In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=d6f018e89a203e48b5a1e761b7e58f5382a6d6e607299126b3a090b723f2f7c3
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F


spark = SparkSession.builder\
        .master('local[4]')\
        .appName('Lesson_2')\
        .config('spark.ui.port', '4050')\
        .config('spark.executor.instances', 2)\
        .config('spark.executor.memory', '5g')\
        .config('spark.executor.cores', 2)\
        .getOrCreate()

sc = spark.sparkContext

### Прочитайте содержимое каждого файла в датафрейм и выведите первые несколько строк
### Применив cast к колонке можно изменить тип данных (F.col().cast(IntegerType()).alias())
### Приведите все данные к правильному типу, либо считайте сразу верно

In [3]:
train = spark.read.csv('train.csv', header=True)
test = spark.read.csv('test.csv', header=True)

In [4]:
train.show(3), test.show(3)

+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|price_range|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|          842|   0|        2.2|       0|  1|     0|         7|  0.6|      188|      2|  2|       20|     756|2549|   9|   7|       19|      0|           0|   1|          1|
|         1021|   1|        0.5|       1|  0|     1|        53|  0.7|      136|      3|  6|      905|    1988|2631|  17|   3|        7|      1|           1|   0|          2|
|          563|   1|        0.5|       1|  2|     1|        41|  0.9|      145|      5|  6|     1263|    1716|2603|  11|   2|     

(None, None)

In [5]:
train.printSchema(), test.printSchema()

root
 |-- battery_power: string (nullable = true)
 |-- blue: string (nullable = true)
 |-- clock_speed: string (nullable = true)
 |-- dual_sim: string (nullable = true)
 |-- fc: string (nullable = true)
 |-- four_g: string (nullable = true)
 |-- int_memory: string (nullable = true)
 |-- m_dep: string (nullable = true)
 |-- mobile_wt: string (nullable = true)
 |-- n_cores: string (nullable = true)
 |-- pc: string (nullable = true)
 |-- px_height: string (nullable = true)
 |-- px_width: string (nullable = true)
 |-- ram: string (nullable = true)
 |-- sc_h: string (nullable = true)
 |-- sc_w: string (nullable = true)
 |-- talk_time: string (nullable = true)
 |-- three_g: string (nullable = true)
 |-- touch_screen: string (nullable = true)
 |-- wifi: string (nullable = true)
 |-- price_range: string (nullable = true)

root
 |-- id: string (nullable = true)
 |-- battery_power: string (nullable = true)
 |-- blue: string (nullable = true)
 |-- clock_speed: string (nullable = true)
 |-- dual_s

(None, None)

# Объедините train и test

In [6]:
#data = train.drop('price_range').unionByName(test.drop('id'))
import pandas as pd

train = train.drop('price_range')
test = test.drop('id')

merged_pandas_df = pd.concat([train.toPandas(), test.toPandas()], axis=0)

data = spark.createDataFrame(merged_pandas_df)

## Рассчитайте соотношение сторон телефона и экрана
### Выведите отсортированное распределение плотности пикселей в пикселях на дюйм

In [7]:
#Добавим столбец "ppi" с рассчитанной плотностью пикселей
from pyspark.sql.functions import round

data = data\
    .withColumn('ppi',
        round((F.pow(F.pow(F.col('px_height'), F.lit(2)) + F.pow(F.col('px_width'), F.lit(2)), F.lit(0.5)) /
        ((F.pow(F.pow(F.col('sc_h'), F.lit(2)) + F.pow(F.col('sc_w'), F.lit(2)), F.lit(0.5))) / 2.54)))
    )
data.show(5)

+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|  ppi|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----+
|          842|   0|        2.2|       0|  1|     0|         7|  0.6|      188|      2|  2|       20|     756|2549|   9|   7|       19|      0|           0|   1|168.0|
|         1021|   1|        0.5|       1|  0|     1|        53|  0.7|      136|      3|  6|      905|    1988|2631|  17|   3|        7|      1|           1|   0|321.0|
|          563|   1|        0.5|       1|  2|     1|        41|  0.9|      145|      5|  6|     1263|    1716|2603|  11|   2|        9|      1|           1|   0

In [8]:
from pyspark.sql.functions import desc

# Выбор показателя для анализа
indicator_column = "ppi"

# Получение распределения показателя в датафрейме
distribution = data.groupBy(indicator_column).count().orderBy(desc("count"))

# Вывод отсортированного распределения показателя
distribution.show()

+-----+-----+
|  ppi|count|
+-----+-----+
|209.0|   18|
|225.0|   17|
|292.0|   16|
|258.0|   15|
|197.0|   15|
|138.0|   15|
|199.0|   14|
|214.0|   13|
|246.0|   13|
|122.0|   13|
|285.0|   13|
|242.0|   13|
|176.0|   13|
|188.0|   13|
|265.0|   13|
|180.0|   12|
|249.0|   12|
|210.0|   12|
|205.0|   12|
|171.0|   12|
+-----+-----+
only showing top 20 rows



In [9]:
#Отсортируем датафрейм по плотности пикселей
data.sort('ppi').show(10)

+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+----+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi| ppi|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+----+
|         1314|   1|        2.8|       1| 13|     1|         5|  1.0|       87|      4| 18|       82|     512|3411|  17|  16|       18|      1|           0|   0|56.0|
|         1088|   1|          3|       1|  2|     1|        52|  0.3|      164|      7|  4|       86|     588|1447|  19|  16|       15|      1|           1|   1|61.0|
|         1203|   1|        0.5|       1|  0|     1|        11|  0.9|      109|      2| 12|       35|     510|1672|  17|  13|       19|      1|           1|   0|61.0

### Сохраните результат в csv sep=';', encoding='cp1251' с колонками id, плотность пикселей и общей инфой в формате "2 ядра 2 гига"

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat, lit

#Добавим столбцы "id" и "Общая информация" в формате "2 ядра 2 гига"
data = data\
        .withColumn('id', F.monotonically_increasing_id())\
        .withColumn("Общая информация", concat(col("n_cores"), lit(" ядра "), col("ram"), lit(" гига")))

#Сохраним данные в файл
data\
.select(
    'id', 
    'ppi', 
    'Общая информация')\
.write.option('encoding', 'cp1252').csv('results.csv', mode='overwrite', sep=';', header=True)

In [11]:
results = spark.read.csv('results.csv', sep=';', header=True)
results.show(5)

+-----------+------+----------------+
|         id|   ppi|????? ??????????|
+-----------+------+----------------+
|17179869184| 197.0|2 ???? 2423 ????|
|17179869185| 318.0|2 ???? 1658 ????|
|17179869186|  79.0|8 ???? 2361 ????|
|17179869187| 174.0|4 ???? 1380 ????|
|17179869188|1069.0|6 ???? 3892 ????|
+-----------+------+----------------+
only showing top 5 rows



In [12]:
#Перезапишем в формате utf8
data\
.select(
    'id', 
    'ppi', 
    'Общая информация')\
.write.option('encoding', 'utf8').csv('results.csv', mode='overwrite', sep=';', header=True)

In [13]:
results = spark.read.csv('results.csv', sep=';', header=True)
results.show(5)

+-----------+------+----------------+
|         id|   ppi|Общая информация|
+-----------+------+----------------+
|17179869184| 197.0|2 ядра 2423 гига|
|17179869185| 318.0|2 ядра 1658 гига|
|17179869186|  79.0|8 ядра 2361 гига|
|17179869187| 174.0|4 ядра 1380 гига|
|17179869188|1069.0|6 ядра 3892 гига|
+-----------+------+----------------+
only showing top 5 rows

